In [1]:
!python -V

Python 3.10.15


In [10]:
!pip list | grep -E "tf|tensorflow|keras|typing"

keras                        2.15.0
keras-core                   0.1.7
keras-nlp                    0.6.4
platformdirs                 4.2.0
tensorflow                   2.15.1
tensorflow-datasets          4.9.6
tensorflow-estimator         2.15.0
tensorflow-hub               0.16.1
tensorflow-io-gcs-filesystem 0.37.1
tensorflow-metadata          1.16.1
tensorflow-text              2.15.0
tf_keras                     2.15.1


In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # "jax" or "tensorflow" or "torch"
import keras_nlp
import tensorflow_datasets as tfds
import keras

2024-11-15 09:36:28.658940: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 09:36:28.692678: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 09:36:28.692704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 09:36:28.694111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 09:36:28.700420: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 09:36:28.700950: I tensorflow/core/platform/cpu_feature_guard.cc:1

Using TensorFlow backend


In [3]:
imdb_train, imdb_test = tfds.load(
    "imdb_reviews",
    split=["train", "test"],
    as_supervised=True,
    batch_size=16,
)

In [4]:
# Load a model.
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
    activation="softmax",
)
# Compile the model.
classifier.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(5e-5),
    metrics=["sparse_categorical_accuracy"],
    jit_compile=True,
)
# Fine-tune.
classifier.fit(imdb_train.take(100), validation_data=imdb_test.take(100))
# Predict new examples.
classifier.predict(["What an amazing movie!", "A total waste of my time."])

17602216/17602216 [==============================] - 2s 0us/step


2024-11-15 09:36:58.190025: I external/local_xla/xla/service/service.cc:168] XLA service 0x577f51611c80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-11-15 09:36:58.190065: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-11-15 09:36:58.323392: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-15 09:36:58.479290: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:59] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. bert_classifier/bert_backbone/embeddings_dropout/dropout/random_uniform/RandomUniform
I0000 00:00:1731663422.405022     805 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-11-1

 67/100 [===================>..........] - ETA: 14s - loss: 0.6823 - sparse_categorical_accuracy: 0.5625

2024-11-15 09:37:32.539770: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


100/100 [==============================] - ETA: 0s - loss: 0.6779 - sparse_categorical_accuracy: 0.5781

2024-11-15 09:37:56.160132: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1/1 [==============================] - 1s 1s/step


array([[0.497544  , 0.502456  ],
       [0.5816187 , 0.41838127]], dtype=float32)

In [5]:
import tensorflow as tf
# Save the model
model_export_path = "keras_bert_imdb/1"

In [6]:
def preprocess(inputs):
    # Convert input strings to token IDs, padding mask, and segment IDs
    preprocessor = classifier.preprocessor
    print("type(preprocessor) is", type(preprocessor))
    encoded = preprocessor(inputs)
    return {
        'token_ids': encoded['token_ids'],
        'padding_mask': encoded['padding_mask'],
        'segment_ids': encoded['segment_ids']
    }

@tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string)])
def serving_fn(inputs):
    preprocessed = preprocess(inputs)
    outputs = classifier(preprocessed)
    return outputs

# Save the model
tf.saved_model.save(
    classifier,
    export_dir=model_export_path,
    signatures={"serving_default": serving_fn}
)

print(f"Model saved to: {model_export_path}")

type(preprocessor) is <class 'keras_nlp.src.models.bert.bert_preprocessor.BertPreprocessor'>
INFO:tensorflow:Assets written to: keras_bert_imdb/1/assets


INFO:tensorflow:Assets written to: keras_bert_imdb/1/assets


Model saved to: keras_bert_imdb/1


In [7]:
!saved_model_cli show --dir keras_bert_imdb/1 --tag_set serve --signature_def serving_default

2024-11-15 09:38:16.859556: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 09:38:16.891245: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 09:38:16.891287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 09:38:16.892363: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 09:38:16.897761: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 09:38:16.897984: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [9]:
!tar cvzf keras_bert_imdb.tar.gz keras_bert_imdb

keras_bert_imdb/
keras_bert_imdb/1/
keras_bert_imdb/1/assets/
keras_bert_imdb/1/variables/
keras_bert_imdb/1/variables/variables.index
keras_bert_imdb/1/variables/variables.data-00000-of-00001
keras_bert_imdb/1/fingerprint.pb
keras_bert_imdb/1/saved_model.pb
